# Projeto Spark Code

Esse projeto visa trazer informações de emprestimos bancários.

Utilizando ferramentas atualizadas e importantes no mercado de tecnologia.
Entre elas:
- Apache Spark
- Google Colab
- Google Drive
- GCP 

### Documentações Utilizadas:

[GCP para Big Data](https://cloud.google.com/architecture/build-a-data-lake-on-gcp?hl=pt-br#cloud-storage-as-data-lake)

### Fontes Utilizadas:
- https://calculojuridico.com.br/como-calcular-emprestimos-financiamentos/
- https://www.infomoney.com.br/planilhas/calcular-emprestimos/
- https://ninjadoexcel.com.br/planilha-amortizacao-emprestimo-excel/



## Configurações

In [1]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

# Import's

In [54]:
from pyspark.sql.functions import col, upper
import pandas as pd

## Download e extração dos arquivos do ngrok

Link de apoio: (https://dashboard.ngrok.com/get-started/setup)

In [4]:
# !wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

### Configuração do Authtoken obtido no site do ngrok

In [5]:
# get_ipython().system_raw('./ngrok authtoken 2PsAql4UytDT4URhHVThKcBvwin_2PAPWHdDtoPoZRqRXi7bg')
# get_ipython().system_raw('./ngrok http 80 &')

### Verificando se a autenticação e a porta funciona

In [6]:
# !curl -s http://localhost:4040/api/tunnels

# Realizando conexão com Google Drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Sessão SparkSession

Uma SparkSession pode ser usada para criar DataFrames, registrar DataFrames como tabelas, executar comandos SQL sobre tabelas, armazenar tabelas em cache e ler arquivos em diferentes formatos.

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

# .config('spark.ui.port', '80') \

# Realizando a leitura dos dados

Para arquivos .zip

In [9]:
# import zipfile

# zipfile.ZipFile('path_arquivo', 'r').extractall('path_destino')

Leitura de CSV

In [13]:
path = '/content/drive/MyDrive/data_sources/emprestimo.csv'
# df = spark.read.csv(path, sep=';', inferSchema=True)
df = spark.read.csv(path, sep=';', header='true')

# Visualização com pandas
Para uma melhor interface, preferi deixar com a visualização em pandas, mas poderia ter feito somente:

`df.show()`

In [65]:
df.toPandas()

,CODIGO_CONTRATO,PRODUTO,NOME,CPF,DESCRICAO_MOVIMENTO,DATA_CONTRATO,DATA_PAGAMENTO,DATA_MOVIMENTACAO,DATA_VENCIMENTO,TAXA_CDI,VALOR_INICIAL,VALOR_PARCELA,VALOR_TOTAL
0,100,CONSIGNADO,Maria Silva,111.111.111-11,None,02/02/2023,5,None,02/02/2024,0,"R$ 22.100,00",0,0
1,101,CONSIGNADO,João Santos,222.222.222-22,None,05/02/2023,5,None,05/02/2024,0,"R$ 17.400,00",0,0
2,102,CONSIGNADO,Ana Costa,333.333.333-33,None,08/02/2023,5,None,08/02/2024,0,"R$ 11.300,00",0,0
3,103,CONSIGNADO,Pedro Oliveira,444.444.444-44,None,22/02/2023,5,None,22/02/2024,0,"R$ 16.700,00",0,0
4,104,CONSIGNADO,Sofia Pereira,555.555.555-55,None,25/02/2023,5,None,25/02/2024,0,"R$ 24.900,00",0,0
5,105,CONSIGNADO,André Almeida,666.666.666-66,None,01/03/2023,5,None,01/03/2024,0,"R$ 13.200,00",0,0
6,106,CRÉDITO PESSOAL,Laura Ferreira,777.777.777-77,None,07/03/2023,5,None,07/03/2024,0,"R$ 19.800,00",0,0
7,107,CRÉDITO PESSOAL,Tiago Rodrigues,888.888.888-88,None,12/03/2023,5,None,12/03/2025,0,"R$ 27.500,00",0,0
8,108,CRÉDITO PESSOAL,Carolina Sousa,999.999.999-99,None,24/03/2023,7,None,24/03/2025,0,"R$ 8.400,00",0,0
9,109,CRÉDITO PESSOAL,Miguel Gomes,000.000.000-00,None,28/03/2023,7,None,28/03/2025,0,"R$ 12.600,00",0,0


# Estatisticas do Dataframe

In [67]:
df.summary().toPandas()

,summary,CODIGO_CONTRATO,PRODUTO,NOME,CPF,DESCRICAO_MOVIMENTO,DATA_CONTRATO,DATA_PAGAMENTO,DATA_MOVIMENTACAO,DATA_VENCIMENTO,TAXA_CDI,VALOR_INICIAL,VALOR_PARCELA,VALOR_TOTAL
0,count,22,22,22,22,0,22,22,0,22,22,22,22,22
1,mean,110.5,None,None,None,None,None,8.636363636363637,None,None,0.0,None,0.0,0.0
2,stddev,6.493586579592718,None,None,None,None,None,3.9586608431382033,None,None,0.0,None,0.0,0.0
3,min,100,ANTECIPAÇÃO 13,Ana Costa,000.000.000-00,None,01/03/2023,10,None,01/03/2024,0,"R$ 10.900,00",0,0
4,25%,105.0,None,None,None,None,None,5.0,None,None,0.0,None,0.0,0.0
5,50%,110.0,None,None,None,None,None,7.0,None,None,0.0,None,0.0,0.0
6,75%,116.0,None,None,None,None,None,10.0,None,None,0.0,None,0.0,0.0
7,max,121,RESTITUIÇÃO IR,Tiago Rodrigues,999.999.999-99,None,30/05/2023,7,None,30/05/2026,0,"R$ 9.600,00",0,0


# Manipulações e cálculos com Pyspark

# Comandos muito interessantes para você aprender Pyspark!

Fazendo o df.toPandas() ele não muda o type para dataframe em pandas, ele permanece em pyspark, apenas é a visualização

In [20]:
type(df)

pyspark.sql.dataframe.DataFrame

Formas para selecionar todas as colunas:

In [30]:
df.select(df.columns)

DataFrame[CODIGO_CONTRATO: string, PRODUTO: string, NOME: string, CPF: string, DESCRICAO_MOVIMENTO: string, DATA_CONTRATO: string, DATA_PAGAMENTO: string, DATA_MOVIMENTACAO: string, DATA_VENCIMENTO: string, TAXA_CDI: string, VALOR_INICIAL: string, VALOR_PARCELA: string, VALOR_TOTAL: string]

In [39]:
df.select('*')

DataFrame[CODIGO_CONTRATO: string, PRODUTO: string, NOME: string, CPF: string, DESCRICAO_MOVIMENTO: string, DATA_CONTRATO: string, DATA_PAGAMENTO: string, DATA_MOVIMENTACAO: string, DATA_VENCIMENTO: string, TAXA_CDI: string, VALOR_INICIAL: string, VALOR_PARCELA: string, VALOR_TOTAL: string]

Ordenando de forma crescente e visualizando no pandas

In [43]:
df\
  .select('*')\
  .orderBy(['CODIGO_CONTRATO'], ascending=True)\
  .limit(5)\
  .toPandas()

,CODIGO_CONTRATO,PRODUTO,NOME,CPF,DESCRICAO_MOVIMENTO,DATA_CONTRATO,DATA_PAGAMENTO,DATA_MOVIMENTACAO,DATA_VENCIMENTO,TAXA_CDI,VALOR_INICIAL,VALOR_PARCELA,VALOR_TOTAL
0,100,CONSIGNADO,Maria Silva,111.111.111-11,None,02/02/2023,5,None,02/02/2024,0,"R$ 22.100,00",0,0
1,101,CONSIGNADO,João Santos,222.222.222-22,None,05/02/2023,5,None,05/02/2024,0,"R$ 17.400,00",0,0
2,102,CONSIGNADO,Ana Costa,333.333.333-33,None,08/02/2023,5,None,08/02/2024,0,"R$ 11.300,00",0,0
3,103,CONSIGNADO,Pedro Oliveira,444.444.444-44,None,22/02/2023,5,None,22/02/2024,0,"R$ 16.700,00",0,0
4,104,CONSIGNADO,Sofia Pereira,555.555.555-55,None,25/02/2023,5,None,25/02/2024,0,"R$ 24.900,00",0,0


Distinct columns

In [45]:
df.select("PRODUTO").distinct().toPandas()

,PRODUTO
0,CRÉDITO HABITACIONAL
1,CONSIGNADO
2,CONSIGNADO PRIVADO
3,RESTITUIÇÃO IR
4,CRÉDITO PESSOAL
5,CRÉDITO ESTUDANTIL
6,ANTECIPAÇÃO 13


Utilizando filter e método que obtém registros, declarando o nome inicial e o nome final

In [52]:
df\
  .filter(col("PRODUTO").startswith("CONSIGNADO"))\
  .filter(col("PRODUTO").endswith("PRIVADO"))\
  .limit(10)\
  .toPandas()

,CODIGO_CONTRATO,PRODUTO,NOME,CPF,DESCRICAO_MOVIMENTO,DATA_CONTRATO,DATA_PAGAMENTO,DATA_MOVIMENTACAO,DATA_VENCIMENTO,TAXA_CDI,VALOR_INICIAL,VALOR_PARCELA,VALOR_TOTAL
0,115,CONSIGNADO PRIVADO,Gabriel Barbosa,678.678.678-67,None,02/05/2023,10,None,02/05/2026,0,"R$ 25.200,00",0,0


Fazendo um upper para as descrições que foram encontradas no filter utilizando LIKE

In [64]:
df\
  .where(df.PRODUTO.like('%IR'))\
  .toPandas()

,CODIGO_CONTRATO,PRODUTO,NOME,CPF,DESCRICAO_MOVIMENTO,DATA_CONTRATO,DATA_PAGAMENTO,DATA_MOVIMENTACAO,DATA_VENCIMENTO,TAXA_CDI,VALOR_INICIAL,VALOR_PARCELA,VALOR_TOTAL
0,110,RESTITUIÇÃO IR,Beatriz Carvalho,123.123.123-12,None,05/04/2023,7,None,05/04/2025,0,"R$ 21.300,00",0,0
1,111,RESTITUIÇÃO IR,Rafael Martins,234.234.234-23,None,14/04/2023,7,None,14/04/2025,0,"R$ 15.100,00",0,0
2,120,RESTITUIÇÃO IR,Leonor Neves,987.987.987-98,None,29/05/2023,15,None,29/05/2026,0,"R$ 18.900,00",0,0


In [62]:
df\
  .where(df.PRODUTO.like('%CRÉDITO%'))\
  .toPandas()

,CODIGO_CONTRATO,PRODUTO,NOME,CPF,DESCRICAO_MOVIMENTO,DATA_CONTRATO,DATA_PAGAMENTO,DATA_MOVIMENTACAO,DATA_VENCIMENTO,TAXA_CDI,VALOR_INICIAL,VALOR_PARCELA,VALOR_TOTAL
0,106,CRÉDITO PESSOAL,Laura Ferreira,777.777.777-77,None,07/03/2023,5,None,07/03/2024,0,"R$ 19.800,00",0,0
1,107,CRÉDITO PESSOAL,Tiago Rodrigues,888.888.888-88,None,12/03/2023,5,None,12/03/2025,0,"R$ 27.500,00",0,0
2,108,CRÉDITO PESSOAL,Carolina Sousa,999.999.999-99,None,24/03/2023,7,None,24/03/2025,0,"R$ 8.400,00",0,0
3,109,CRÉDITO PESSOAL,Miguel Gomes,000.000.000-00,None,28/03/2023,7,None,28/03/2025,0,"R$ 12.600,00",0,0
4,112,CRÉDITO ESTUDANTIL,Mariana Ramos,345.345.345-34,None,15/04/2023,7,None,15/04/2025,0,"R$ 23.700,00",0,0
5,113,CRÉDITO ESTUDANTIL,Guilherme Castro,456.456.456-45,None,23/04/2023,10,None,23/04/2026,0,"R$ 10.900,00",0,0
6,114,CRÉDITO HABITACIONAL,Inês Cardoso,567.567.567-56,None,27/04/2023,10,None,27/04/2026,0,"R$ 16.400,00",0,0
7,116,CRÉDITO PESSOAL,Luísa Fernandes,789.789.789-78,None,10/05/2023,10,None,10/05/2026,0,"R$ 9.600,00",0,0
8,119,CRÉDITO HABITACIONAL,Diogo Lima,098.098.098-09,None,27/05/2023,15,None,27/05/2026,0,"R$ 11.800,00",0,0
9,121,CRÉDITO PESSOAL,José Bezerra,876.876.876-87,None,30/05/2023,15,None,30/05/2026,0,"R$ 27.200,00",0,0
